## 1. 사용할 패키지 불러오기

In [2]:
import pandas as pd
import cv2
import os
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.nn import L1Loss
import torch.nn.functional as F
from data_gen.data_gen import DatasetGenerator
import torch
import torchvision.models as models
import torch.nn as nn
import numpy as np
from tqdm import tqdm


/home/jongwook95.lee/miniforge3/envs/vision_machine/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. 데이터 불러오기

In [ ]:
final_data = pd.read_excel('final_data.xlsx')
final_data.head()

## 3. 이미지 별 RGB 평균 및 HSV 평균 변수 추가

In [ ]:
final_data['R'] = 0
final_data['G'] = 0
final_data['B'] = 0
final_data['H'] = 0
final_data['S'] = 0
final_data['V'] = 0

for i in range(len(final_data['이미지경로'])):
    if i % 500 == 0:
        print(i)
    bgr = cv2.imread(final_data['이미지경로'][i])
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    

    final_data['R'][i] = np.mean(bgr[:, :, 2][bgr[:, :, 2] != 255])
    final_data['G'][i] = np.mean(bgr[:, :, 1][bgr[:, :, 1] != 255])
    final_data['B'][i] = np.mean(bgr[:, :, 0][bgr[:, :, 0] != 255])
    final_data['H'][i] = np.mean(hsv[:, :, 0][hsv[:, :, 0] != 255])
    final_data['S'][i] = np.mean(hsv[:, :, 1][hsv[:, :, 1] != 255])
    final_data['V'][i] = np.mean(hsv[:, :, 2][hsv[:, :, 2] != 255])
    

In [ ]:
final_data.to_excel('final_data_rgb_hsv.xlsx', index = False, encoding = 'euc-kr')

## 4. 이미지외 변수 및 이미지 모두활용한 분석

### (1) Dataset 생성

In [3]:
final_data = pd.read_excel('final_data_rgb_hsv.xlsx')

In [4]:
all_variable = ['작가생존여부_사망', '작가생존여부_생존', '작가생존여부_알수없음', '판매계절_가을', '판매계절_겨울', '판매계절_봄',
       '판매계절_여름', '재료_견본채색', '재료_기타', '재료_브론즈', '재료_비단에수묵담채', '재료_석판화',
       '재료_실크스크린', '재료_알수없음', '재료_오프셋석판화', '재료_종에이수묵담채', '재료_종이에먹', '재료_종이에수묵',
       '재료_종이에수묵담채', '재료_종이에수묵채색', '재료_종이에수채', '재료_종이에유채', '재료_지본묵서',
       '재료_지본수묵', '재료_지본채색', '재료_캔버스에아크릴', '재료_캔버스에유채', '재료_캔버스에혼합재료',
       '판매처_꼬모옥션', '판매처_마이아트옥션', '판매처_서울옥션', '판매처_아이옥션', '판매처_에이옥션', '판매처_칸옥션',
       '판매처_케이옥션', '판매처_헤럴드아트데이', '가로', '세로', '작품 판매 횟수', '이미지경로', 'R', 'G', 'B', 'H', 'S', 'V']

table_variable = ['작가생존여부_사망', '작가생존여부_생존', '작가생존여부_알수없음', '판매계절_가을', '판매계절_겨울', '판매계절_봄',
       '판매계절_여름', '재료_견본채색', '재료_기타', '재료_브론즈', '재료_비단에수묵담채', '재료_석판화',
       '재료_실크스크린', '재료_알수없음', '재료_오프셋석판화', '재료_종에이수묵담채', '재료_종이에먹', '재료_종이에수묵',
       '재료_종이에수묵담채', '재료_종이에수묵채색', '재료_종이에수채', '재료_종이에유채', '재료_지본묵서',
       '재료_지본수묵', '재료_지본채색', '재료_캔버스에아크릴', '재료_캔버스에유채', '재료_캔버스에혼합재료',
       '판매처_꼬모옥션', '판매처_마이아트옥션', '판매처_서울옥션', '판매처_아이옥션', '판매처_에이옥션', '판매처_칸옥션',
       '판매처_케이옥션', '판매처_헤럴드아트데이', '가로', '세로', '작품 판매 횟수', 'R', 'G', 'B', 'H', 'S', 'V']

In [5]:
X = final_data.loc[:, all_variable]
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
X.loc[:, ['R', 'G', 'B', 'H', 'S', 'V']] = min_max_scaler.fit_transform(X.loc[:, ['R', 'G', 'B', 'H', 'S', 'V']] )
X.head()


y = np.log10(final_data['판매가격'])

In [6]:
train_dataset, test_dataset, train_target, test_target = train_test_split(X, y, train_size = 0.8, random_state = 1004)

### (2) 이미지 데이터와 이미지외 변수 데이터로 분할

In [7]:
train_image = train_dataset['이미지경로']
train_table = train_dataset.loc[:, table_variable]
train_table = np.array(train_table)

test_image = test_dataset['이미지경로']
test_table = test_dataset.loc[:, table_variable]
test_table = np.array(test_table)

In [8]:
train_image_dataset_generator = DatasetGenerator(list(train_image), list(train_target), batch_size = 16, phase = 'train', train_valid_split = False)
train_dataloader = train_image_dataset_generator.dataloader()

test_image_dataset_generator = DatasetGenerator(list(test_image), list(test_target), batch_size = 1, phase = 'test', train_valid_split = False)
test_dataloader = test_image_dataset_generator.dataloader()

### (4) Model 생성

In [9]:
import torch
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo


__all__ = ['ResNet', 'resnet18_cbam', 'resnet34_cbam', 'resnet50_cbam', 'resnet101_cbam',
           'resnet152_cbam']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
           
        self.fc = nn.Sequential(nn.Conv2d(in_planes, in_planes // 16, 1, bias=False),
                               nn.ReLU(),
                               nn.Conv2d(in_planes // 16, in_planes, 1, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()

        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out = self.ca(out) * out
        out = self.sa(out) * out

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)

        self.ca = ChannelAttention(planes * 4)
        self.sa = SpatialAttention()

        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out = self.ca(out) * out
        out = self.sa(out) * out

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [10]:
def resnet18_cbam(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        pretrained_state_dict = model_zoo.load_url(model_urls['resnet18'])
        now_state_dict        = model.state_dict()
        now_state_dict.update(pretrained_state_dict)
        model.load_state_dict(now_state_dict)
    return model

In [11]:
# build model
vision_model = resnet18_cbam(pretrained=True)
num_ftrs = vision_model.fc.in_features
vision_model.fc = nn.Linear(num_ftrs, 1)

### (6) Load Best Model

In [12]:
vision_model.load_state_dict(torch.load('result/Best_image_resnet18_cbam_model.pth'))

<All keys matched successfully>

In [13]:
vision_model.fc = nn.Identity()

### (7) 성능 평가

#### - 학습 데이터에 생성

In [16]:
train_image_dataset_generator = DatasetGenerator(list(train_image), list(train_target), batch_size = 1, phase = 'test', train_valid_split = False)
train_dataloader = train_image_dataset_generator.dataloader()

In [17]:
print('start prediction')
predictions = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vision_model.to(device)

with torch.no_grad():  
    for data in train_dataloader['test']:
        images, labels = data['image'].float().to(device),  data['target'].float().to(device)
        vision_model.eval()  
        yhat = vision_model(images)  
        pred = list(yhat.cpu().numpy())
        predictions.append(pred)

start prediction


In [18]:
predictions = np.stack(predictions)[:, 0, :]
X_train = pd.DataFrame(np.hstack([train_table, predictions]))
y_train = train_target

#### - 테스트 데이터 생성

In [19]:
print('start prediction')
predictions = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vision_model.to(device)

with torch.no_grad():  
    for data in test_dataloader['test']:
        images, labels = data['image'].float().to(device),  data['target'].float().to(device)
        vision_model.eval()  
        yhat = vision_model(images)  
        pred = list(yhat.cpu().numpy())
        predictions.append(pred)

start prediction


In [20]:
predictions = np.stack(predictions)[:, 0, :]
X_test = pd.DataFrame(np.hstack([test_table, predictions]))
y_test = test_target

#### - XGBoost Modeling

In [22]:
from xgboost import XGBRegressor
n_estimators_candidate = [100, 300, 500]
max_depth_candidate = [3, 5, 7, 9]
learning_rate_candidate = [0.1, 0.01, 0.001]

# 결과를 저장할 빈 리스트 생성
n_estimators_list = []
max_depth_list = []
learning_rate_list = []
train_score_list = []
val_score_list = []

# XGBoost의 n_estimators 파라미터에 대해서
for n_estimators in n_estimators_candidate:
    # XGBoost의 max_depth 파라미터에 대해서
    for max_depth in max_depth_candidate:
        # XGBoost의 learning_rate 파라미터에 대해서
        for learning_rate in learning_rate_candidate:
            # 모델 생성 및 학습
            model = XGBRegressor(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, objective='reg:squarederror').fit(X_train, y_train)
            # Train 데이터에 대한 결과 확인
            train_predict = model.predict(X_train)
            train_score_list.append(np.sqrt(mean_squared_error(y_train, train_predict)))
            # Test 데이터에 대한 결과 확인
            test_predict = model.predict(X_test)
            val_score_list.append(np.sqrt(mean_squared_error(y_test, test_predict)))
            # Parameter 저장
            n_estimators_list.append(n_estimators)
            max_depth_list.append(max_depth)
            learning_rate_list.append(learning_rate)
            
result = pd.DataFrame({"n_estimators": n_estimators_list, "max_depth": max_depth_list, "learning_rate": learning_rate_list, 'Train Score': train_score_list, 'Test Score': val_score_list})
result = result.loc[result['Test Score'] == min(result['Test Score']), :].reset_index(drop = True)
xgb_model = XGBRegressor(n_estimators=result['n_estimators'][0], max_depth=result['max_depth'][0], learning_rate=result['learning_rate'][0], objective='reg:squarederror').fit(train_X, train_y)
result

KeyboardInterrupt: 

In [24]:
result = pd.DataFrame({"n_estimators": n_estimators_list, "max_depth": max_depth_list, "learning_rate": learning_rate_list, 'Train Score': train_score_list, 'Test Score': val_score_list})
result = result.loc[result['Test Score'] == min(result['Test Score']), :].reset_index(drop = True)
xgb_model = XGBRegressor(n_estimators=result['n_estimators'][0], max_depth=result['max_depth'][0], learning_rate=result['learning_rate'][0], objective='reg:squarederror').fit(X_train, y_train)
result

,n_estimators,max_depth,learning_rate,Train Score,Test Score
0,100,7,0.1,0.380034,0.551494


In [25]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
train_predict = xgb_model.predict(X_train)
print("Train RMSE: {}".format(np.sqrt(mean_squared_error(y_train, train_predict))))
print("Train R2 Score: {}".format(r2_score(y_train, train_predict)))
print("Train MAE: {}".format(np.sqrt(mean_absolute_error(y_train, train_predict))))
print("Train MAPE: {}".format(mean_absolute_percentage_error(y_train, train_predict)))

Train RMSE: 0.38003439677666156
Train R2 Score: 0.6877637221225745
Train MAE: 0.5370621540857569
Train MAPE: 0.04769556104938614


In [26]:
from sklearn.metrics import mean_squared_error, r2_score
test_predict = xgb_model.predict(X_test)
print("Test RMSE: {}".format(np.sqrt(mean_squared_error(y_test, test_predict))))
print("Test R2 Score: {}".format(r2_score(y_test, test_predict)))
print("Test MAE: {}".format(mean_absolute_error(y_test, test_predict)))
print("Test MAPE: {}".format(mean_absolute_percentage_error(y_test, test_predict)))

Test RMSE: 0.5514944916823039
Test R2 Score: 0.3798682938177199
Test MAE: 0.42601560990056325
Test MAPE: 0.07018551426366483
